## Numerical simulation of the Langevin equation

Pierre de Buyl  
Instituut voor Theoretische Fysica, KU Leuven

The code and notebooks in the repository `2018_nonequilibrium_simulations`
constitute supplementary material for the lecture notes
*Langevin simulations for nonequilibrium physics*.
See the [README.md](README.md) file for more information.
See the lecture notes (link posted soon, visit [my website](http://pdebuyl.be/)
for updates.

This notebook contains a Python implementation of the Euler-Maruyama and
Stochastic Runge-Kutta algorithms and short examples of use.


In [ ]:
# import libraries and set default figure parameters
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.subplot.top'] = 0.95
plt.rcParams['figure.subplot.right'] = 0.95

plt.rcParams['font.size'] = 16

### Implementation of the algorithms

The routines below take as input arguments:
- x: a scalar or a vector
- f: a Python function that returns the force on the particle
- mu: the mobility of the particles
- T: the temperature
- dt: the timestep
- n: the number of iterations to perform

The routines return the value of x afer n steps.

In [ ]:
def euler(x, f, mu, T, dt, n):
    x = np.array(x).copy()
    shape = x.shape
    step = np.sqrt(2*mu*T*dt)
    for i in range(n):
        g0 = np.dot(step, np.random.normal(size=shape))
        f1 = f(x)
        x = x + dt*np.dot(mu, f1) + g0
    return x

def srk(x, f, mu, T, dt, n):
    x = np.array(x).copy()
    shape = x.shape
    step = np.sqrt(2*mu*T*dt)
    for i in range(n):
        g0 = np.dot(step, np.random.normal(size=shape))
        f1 = f(x)
        x1 = x + np.dot(mu, f1) * dt + g0
        f2 = f(x1)
        x = x + 0.5*dt*np.dot(mu, f1+f2) + g0
    return x


### Running a simulation

The algorithms `euler` and `srk` take as input a value for x, a force term,
the mobility of the particles, the temperature, the time step and the number
of iterations.

The algorithms return one point per call, so a typical simulation calls many times
the algorithms in a loop, collecting data as needed.

In [ ]:
# force that returns 0, for Brownian motion
def zero_force(x):
    return 0*x

In [ ]:
x = 0
mu = 21
T = 1/10
dt = 1e-2
nsteps = 50

trajectory = []
for i in range(1000):
    x = euler(x, zero_force, mu, T, dt, nsteps)
    trajectory.append(x)

trajectory = np.array(trajectory)

In [ ]:
plt.figure()

plt.plot(trajectory)
plt.xlabel('time')
plt.ylabel('position')

In [ ]:
mu = 1
T = 1
dt = 3e-2
nsteps = 200
npoints = 100000

def quartic_force(x):
    return -x**3

x = 0
x = euler(x, quartic_force, mu, T, dt, 100*nsteps)

euler_trajectory = []
for i in range(npoints):
    x = euler(x, quartic_force, mu, T, dt, nsteps)
    euler_trajectory.append(x)

euler_trajectory = np.array(euler_trajectory)

x = 0
x = euler(x, quartic_force, mu, T, dt, 100*nsteps)

srk_trajectory = []
for i in range(npoints):
    x = srk(x, quartic_force, mu, T, dt, nsteps)
    srk_trajectory.append(x)

srk_trajectory = np.array(srk_trajectory)

In [ ]:
euler_count, bins, patches = plt.hist(euler_trajectory, bins=25, histtype='step', density=True)

srk_count, _, patches = plt.hist(srk_trajectory, bins=bins, histtype='step', density=True)

bins = (bins[1:]+bins[:-1])/2

eq = np.exp(-bins**4/(4*T))
eq /= (eq.sum()*(bins[1]-bins[0]))

plt.plot(bins, eq);

plt.figure()

plt.plot(bins, euler_count-eq)
plt.plot(bins, srk_count-eq)

### Running many simulations

The study of stochastic models requires typically the analysis of many simulations.
Below, I provide an example of running the same simulation many times and of averaging
the result.

In the example, I compute via the histogram routine the probability distribution of a
particle in a two-dimensional quadratic potential with spring constant 1/4.

In [ ]:
k = 1/4

def harmonic_force(x):
    return -k*x

mu = 1
T = 2
dt = 1e-2
n_steps = 50
n_points = 20000
n_trajectories = 5

all_trajectories = []
for t in range(n_trajectories):
    x = np.array([0, 0])

    x = srk(x, harmonic_force, mu, T, dt, 100*n_steps)

    trajectory = []
    for i in range(n_points):
        x = srk(x, harmonic_force, mu, T, dt, n_steps)
        trajectory.append(x)

    all_trajectories.append(np.array(trajectory))

all_trajectories = np.array(all_trajectories)

In [ ]:
count, x_edges, y_edges = np.histogram2d(all_trajectories[:,:,0].flatten(), all_trajectories[:,:,1].flatten(), bins=10, normed=True)

In [ ]:
plt.contour((x_edges[1:]+x_edges[:-1])/2, (y_edges[1:]+y_edges[:-1])/2, count, [5e-4, 1e-3, 3e-3], colors='k')

X, Y = np.meshgrid(x_edges, y_edges)

Z = np.exp(-k*(X**2+Y**2)/(2*T))/(2*np.pi*T/k)

plt.contour(X, Y, Z, [1e-3, 2e-3, 3e-3], colors='r')

In [ ]:
energy = k*(all_trajectories[:,:,0]**2 + all_trajectories[:,:,1]**2)/2
energy = energy.flatten()

count, bins, _ = plt.hist(energy, density=True, bins=32, histtype='step')

plt.plot(bins, np.exp(-bins/T)/T);

In [ ]:
%load_ext cython

In [ ]:
import algorithms

In [ ]:
%%cython

cimport algorithms
from libc.math cimport cos

cdef class cy_quartic(algorithms.cyfunc_nd):
    cpdef void force(self, double[::1] x, double[::1] f):
        cdef int i
        for i in range(x.shape[0]):
            f[i] = -x[i]**3


In [ ]:
mu = 1
T = 1
dt = 3e-2
nsteps = 200
npoints = 400000

def quartic_force(x):
    return -x**3

x = 0
x = euler(x, quartic_force, mu, T, dt, 100*nsteps)

euler_trajectory = algorithms.integrate_euler(np.array([x]), np.array([1.]), T, dt, npoints, nsteps, f=cy_quartic())

x = 0
x = srk(x, quartic_force, mu, T, dt, 100*nsteps)
srk_trajectory = algorithms.integrate_srk(np.array([x]), np.array([1.]), T, dt, npoints, nsteps, f=cy_quartic())


In [ ]:
plt.figure()
euler_count, bins, patches = plt.hist(euler_trajectory, bins=25, histtype='step', density=True, label='euler')

srk_count, _, patches = plt.hist(srk_trajectory, bins=bins, histtype='step', density=True, label='srk')

bins = (bins[1:]+bins[:-1])/2

eq = np.exp(-bins**4/(4*T))
eq /= (eq.sum()*(bins[1]-bins[0]))

plt.plot(bins, eq, label='eq.');
plt.legend()

plt.figure()

plt.plot(bins, euler_count-eq, label='euler')
plt.plot(bins, srk_count-eq, label='srk')
plt.legend();